# Stacked Regression Approach for Nowcasting Multidimensional Poverty

This notebook implements a three-stage method to nowcast multidimensional poverty for Mexican states using text-derived features. To address the *small n, large p* challenge, the pipeline includes:

- **Feature Selection via LASSO:** this regularized linear model is used to identify a parsimonious subset of relevant predictors. This step mitigates overfitting risks common in high-dimensional, low-sample-size settings.
- **Base Learners:** predictions are generated using **Random Forest Regression** and **Partial Least Squares Regression** (PLS). These models offer complementary strengths: RF captures complex non-linear relationships, while PLS provides stable linear modeling effective in small-sample scenarios.
- **Meta-Learner:** we chose a simple Linear Regression to combine individual model's predictions and generate the final output. 

The rationale for adopting this stacking strategy is rooted in evidence from [Han et al., 2021](https://doi.org/10.1186/s12911-021-01688-3), who show that combining RF and Generalized Linear Models through stacking improves prediction in small-sample, high-dimensional biomedical datasets. Notably, stacking was most effective when the base learners produced dissimilar prediction profiles, allowing the meta-learner to integrate complementary information. Inspired by this, we stack RF and PLS, which are structurally highly distinct, to exploit their modeling capabilities.

## Data Integration

Six sources of text-based indicators are integrated across two years (2020 and 2022):

- **Google Trends**: Search volume patterns for poverty-related keywords
- **YouTube Comments**: Sentiment score and share of comments about each dimension of poverty 
- **Telegram Posts**: Share of posts about each dimension of poverty 
- **News Outlets**: LDA topic shares of media coverage
- **Official Statistics**: CONEVAL poverty measurements (ground truth)

**Geographic Scope**: All 32 Mexican states.  
**Temporal Scope**: Two reference years — 2020 and 2022.

## Feature Engineering and Model Training

- All numeric indicators (except the targets, `state` and `year`) are used as features.
- We are scaling features before implementing LASSO. Random Forest, since it's a tree-based model, would not need scaling, but since it is necessary for all the other models, we scaled features. However, this won't affect Random Forest performance. 
- A separate model is trained for each poverty dimension.

We implemented two training-validation strategies:

1. **In-Sample Setting**: Model trained on both 2020 and 2022 data, validated on 2022.
2. **Out-of-Sample Setting**: Model trained only on 2020, validated on 2022.

## Model Architecture: LASSO + RF + PLS (Stacked)

### 1. **Feature Selection via LASSO Regression**
```python
# Apply LASSO independently for each poverty dimension
lasso = Lasso(
    alpha=1.0,  # Fixed regularization strength
    random_state=42,
    max_iter=10000
).fit(X_scaled, y)

# Select features with non-zero coefficients
selected_mask = lasso.coef_ != 0
selected_features = features[selected_mask]
```

- LASSO is applied independently for each of the six poverty dimensions
- Selects a parsimonious subset of informative predictors from the full feature set (39 variables), driving irrelevant coefficients exactly to zero
- **Fixed α = 1.0**: Provides strong regularization, ensuring aggressive feature selection suitable for high-dimensional, small-sample settings

### 2. **Base Learners**
```python
base_learners = [
    ('rf', RandomForestRegressor(
        n_estimators=100,      # sufficient trees for stable predictions
        max_depth=4,           # prevents overfitting in small samples
        min_samples_leaf=2,    # ensures sufficient observations per leaf
        random_state=42
    )),
    ('pls', PLSRegression(
        n_components=2         # always use two latent components 
    ))
]
```

**Random Forest Regressor**: Captures complex non-linear patterns and feature interactions through ensemble of decision trees. We applied fixed hyperparameters, specifically chosen to prevent overfitting:
  - `max_depth=4`: A shallow tree depth limits the model's ability to fit highly specific patterns that may not generalize beyond the sample. This value was selected to strike a balance between model flexibility and the risk of overfitting on sparse or noisy signals, which are common in high-dimensional text-derived features.
  - `min_samples_leaf=2`: Requiring at least two samples per leaf prevents the model from creating overly specific splits that isolate single observations, which would otherwise increase variance and reduce generalization. While this threshold is low, it is appropriate given the extremely small sample we have. 
  - `n_estimators=100`: Using 100 trees provides sufficient averaging to stabilize predictions without making the model unnecessarily expensive. 

**Partial Least Squares (PLS) Regression**: Well-suited for high-dimensional settings where predictors are highly correlated, as it extracts latent components that summarize the most relevant variation in both predictors and the response. We applied a fixed number of components, equal to 2, across all poverty dimensions. We believe this is appropriate since:
  - **Overfitting control**: Limiting the number of components is critical given the small sample size (32 states), helping prevent the model from fitting noise.
  - **Interpretability**: Two components strike a balance between capturing key socioeconomic variation and maintaining a model structure that can be interpreted across dimensions.
  - **Empirically validated**: We tried with different number of components, and 2 yielded the best results on average. 

### 3. **Stacking Layer with Cross-Validation**
```python
# Create StackingRegressor with automatic CV handling
stacking_regressor = StackingRegressor(
    estimators=base_learners,
    final_estimator=LinearRegression(),
    cv=5,                    # 5-fold CV for meta-learner training only
    n_jobs=-1
)

# Train with automatic out-of-fold prediction generation
stacking_regressor.fit(X_train, y_train)
```

**Key advantages of StackingRegressor**:
- **Automatic cross-validation**: Generates out-of-fold predictions to train the meta-learner, preventing data leakage
- **Robust generalization**: Each base learner is trained on k-1 folds and predicts on the held-out fold
- **Meta-learner training**: Linear regression learns optimal weights to combine base learner predictions

### 4. **Final Nowcasts**

The stacked model produces final predictions by optimally combining the complementary strengths of Random Forest and PLS. We believe that the main two advantages of combining these models are:

- **Balancing Bias and Variance:** Random Forest has low bias but moderate variance:
  - The absence of assumptions about the functional form of the relationship between features and poverty indicators allows it to capture complex interactions -> low bias. 
  - Its predictions are sensitive to small changes in the training data. With only 32 states, different random samples or slight data modifications can lead to substantially different tree structures and predictions -> moderate variance (This is partially overcome given the averaging over multiple trees that RF does, so we don't say 'high variance', typical of simple trees, but 'moderate variance')

   By contrast, PLS has low variance but high bias:
  - PLS is constrained to linear combinations of latent components, so it can potentially miss important patterns when relationships are non-linear -> high bias.
  - PLS produces stable, consistent predictions across different samples. Reducing dimensionality to 2 components makes predictions less sensitive to small changes in training data, which is crucial in small samples -> low variance. 

- **Different functional form:** as already mentioned, PLS captures linear relationships while RF captures non-linear patterns:
    - When poverty-feature relationships are approximately linear, PLS provides reliable estimates while RF might overfit noise
    - When relationships are genuinely non-linear, RF captures these patterns while PLS misses them 

### Why We Avoided Full Hyperparameter Tuning

Given the limited sample size, we adopt a **fixed hyperparameter approach** rather than relying on automatic tuning through cross-validation (CV). Although CV is a standard procedure for model selection, in small-*n* contexts it can introduce more instability than benefit.

We initially experimented with **GridSearchCV** and **RandomizedSearchCV** under various CV strategies, including **Leave-One-Out (LOOCV)**, **K-Fold**, and **Repeated K-Fold**. However, across all configurations, the models tuned via CV consistently performed much **worse** than our fixed-parameter baseline. 

To understand this behavior, we inspected the **variance in performance across CV folds**, and found it to be remarkably high. This high variance indicates that model performance is **sensitive to small changes in training data**.

Indeed, with only 32 training points (we are specifically focusing on the out-of-sample performance as it is the most important one), the model evaluation metric fluctuates substantially due to small differences in training subsets. As a result, hyperparameter search tends to select configurations that **appear optimal due to chance** rather than generalizable performance.

These findings are consistent with [Han et al., 2021](https://doi.org/10.1186/s12911-021-01688-3), who demonstrate that traditional CV-based hyperparameter selection can be **ineffective** in small-samples.

## Model Evaluation

Each model was evaluated on 2022 using:

- **R-squared (R²)**: Measures explained variance.
- **Mean Absolute Error (MAE)**: Captures average prediction error.

Results are stored in:
- `results.csv`: contains actual vs predicted values for each state and dimension.
- `metrics.csv`: includes model performance and the list of important features (with importances).
- Scatter plots: for each poverty dimension, we visualize predicted vs actual values.

In [1]:
# load necessary libraries
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# load the data
tg_2020 = pd.read_csv('clean_data/tg_2020.csv')
tg_2022 = pd.read_csv('clean_data/tg_2022.csv')
gt_2020 = pd.read_csv('clean_data/gt_2020.csv')
gt_2022 = pd.read_csv('clean_data/gt_2022.csv')
yt_2020 = pd.read_csv('clean_data/yt_2020.csv')
yt_2022 = pd.read_csv('clean_data/yt_2022.csv')
news_2020 = pd.read_csv('clean_data/news_2020.csv')
news_2022 = pd.read_csv('clean_data/news_2022.csv')
off_2020 = pd.read_csv('clean_data/off_2020.csv')
off_2022 = pd.read_csv('clean_data/off_2022.csv')

In [3]:
# there were inconsistencies in the state names, so this mapping standardizes the state names across all datasets
state_name_map = {
    "México": "Estado de México",
    "Mexico": "Estado de México",
    "Estados Unidos Mexicanos": "Estado de México",
    "Michoacán de Ocampo": "Michoacán",
    "Veracruz de Ignacio de la Llave": "Veracruz",
    "Coahuila de Zaragoza": "Coahuila",
    "Yucatan": "Yucatán",
    "Queretaro": "Querétaro",
    "San Luis Potosi": "San Luis Potosí",
    "Nuevo Leon": "Nuevo León",
    "Michoacan": "Michoacán",
    "Michoacán de Ocampo": "Michoacán"}

# aplply the mapping
for df in [off_2020, off_2022, gt_2020, gt_2022, yt_2020, yt_2022, tg_2020, tg_2022, news_2020, news_2022]:
    df['state'] = df['state'].astype(str).str.strip()
    df['state'] = df['state'].replace(state_name_map)
    df['state'] = df['state'].replace("nan", None)  

# 'state' columns as strings in all dataframes
for df in [off_2020, off_2022, gt_2020, gt_2022, yt_2020, yt_2022, tg_2020, tg_2022, news_2020, news_2022]:
    df['state'] = df['state'].astype(str)

In [4]:
# create dataset for 2020
data_2020 = off_2020.copy()
data_2020['year'] = 2020

# merge Google Trends
data_2020 = data_2020.merge(gt_2020, on='state', how='inner')

# merge YouTube
data_2020 = data_2020.merge(yt_2020, on='state', how='inner')

# merge Telegram
data_2020 = data_2020.merge(tg_2020, on='state', how='inner')

# merge News (=LDA topics)
data_2020 = data_2020.merge(news_2020, on='state', how='inner')


# create dataset for 2022
data_2022 = off_2022.copy()
data_2022['year'] = 2022

# merge Google Trends
data_2022 = data_2022.merge(gt_2022, on='state', how='inner')

# merge YouTube
data_2022 = data_2022.merge(yt_2022, on='state', how='inner')

# merge Telegram
data_2022 = data_2022.merge(tg_2022, on='state', how='inner')

# merge News (=LDA topics)
data_2022 = data_2022.merge(news_2022, on='state', how='inner')

# combine datasets for 2020 and 2022
combined_data = pd.concat([data_2020, data_2022], ignore_index=True)

# check that we have the correct number of observations in each year
print(f"Number of observations in 2020 dataframe: {len(data_2020)}")
print(f"Number of observations in 2022 dataframe: {len(data_2022)}")

# check that we have all the states in both years (32 states * 2 years = 64 observations)
print(f"Number of observations in combined dataframe: {len(combined_data)}")

Number of observations in 2020 dataframe: 32
Number of observations in 2022 dataframe: 32
Number of observations in combined dataframe: 64


In [5]:
# map poverty dimensions to target columns
POVERTY_DIMENSIONS = {
    'income': 'income_target',
    'health': 'health_target',
    'education': 'educ_target',
    'social_security': 'social_target',
    'housing': 'housing_target',
    'food': 'food_target'}

In [6]:
# get features to use (excluding targets, state, year)
def get_all_feature_columns(data, target_columns, exclude_cols=['state', 'year']):
    return [col for col in data.columns if col not in target_columns + exclude_cols and data[col].dtype in [np.float64, np.int64]]

# define a list of all possible features to use - we do it only once since the features are the same for both years
all_features = get_all_feature_columns(combined_data, list(POVERTY_DIMENSIONS.values()))

print(f" Features available for LASSO selection: {len(all_features)}")

 Features available for LASSO selection: 39


In [7]:
# extract only 2022 data for evaluation - this is identical for both in-sample and out-of-sample evaluation
df_test = combined_data[combined_data['year'] == 2022].copy()
print(f"Test data shape (2022 test set): {df_test.shape}")

Test data shape (2022 test set): (32, 47)


The test sample has 47 columns while with the `get_all_feature_columns` function we selected 39 features, and this is correct since:
- we removed the target columns (1 per dimension, so 6 in total)
- we removed the year and state columns 

47 - 8 = 39

In [8]:
# SELECT FEATURES WITH LASSO USING THE FULL DATASET

# copy the full dataset for scaling
df_full = combined_data.copy()

# standardize features
scaler_global = StandardScaler()
X_full_scaled = scaler_global.fit_transform(df_full[all_features])

# create a new DataFrame with scaled features
df_scaled = pd.DataFrame(X_full_scaled, columns=all_features, index=df_full.index)
# add the target columns AND other necessary columns to the scaled DataFrame
for dim, target_col in POVERTY_DIMENSIONS.items():
    df_scaled[target_col] = df_full[target_col]

# add state and year columns for later filtering
df_scaled['state'] = df_full['state']
df_scaled['year'] = df_full['year']

print(f"Scaled dataset shape: {df_scaled.shape}")

selected_features_in_sample = {}

for dim, target in POVERTY_DIMENSIONS.items():
    print(f"{dim}")
    
    # use scaled data directly
    X_scaled = df_scaled[all_features].values
    y = df_scaled[target].values
    
    # apply LASSO
    lasso = Lasso(
        random_state=42,
        alpha=1,
        max_iter=10000
    ).fit(X_scaled, y)
    
    # select features
    selected_mask = lasso.coef_ != 0
    selected_vars = np.array(all_features)[selected_mask]
    selected_features_in_sample[dim] = selected_vars.tolist()
    
    print(f"Selected {len(selected_vars)} / {len(all_features)} features")

Scaled dataset shape: (64, 47)
income
Selected 15 / 39 features
health
Selected 14 / 39 features
education
Selected 5 / 39 features
social_security
Selected 14 / 39 features
housing
Selected 11 / 39 features
food
Selected 8 / 39 features


In [9]:
# build the stacking model for in-sample validation
results_in_sample = {}
stacking_models_in_sample = {}

for dim, features in selected_features_in_sample.items():
    
    print(f"\nStacking model for {dim}")
    
    # prepare training data (2020+2022 combined)
    X_train = df_scaled[features].values
    y_train = df_scaled[POVERTY_DIMENSIONS[dim]].values
    
    # prepare test data (2022 only) 
    df_test_scaled = df_scaled[df_scaled['year'] == 2022]
    X_test_scaled = df_test_scaled[features].values
    y_test = df_test_scaled[POVERTY_DIMENSIONS[dim]].values
    states = df_test_scaled['state'].values
    
    # define base learners for stacking
    base_learners = [
        ('rf', RandomForestRegressor(n_estimators=100, max_depth=4, min_samples_leaf=2, random_state=42)),
        ('pls', PLSRegression(n_components=2))]
    
    # define meta-model
    meta_model = LinearRegression()
    
    # create StackingRegressor, which automatically handles cross-validation to do 
    # out-of-fold predictions to prevent data leakage
    stacking_regressor = StackingRegressor(
        estimators=base_learners,
        final_estimator=meta_model,
        cv=5, 
        n_jobs=-1)
    
    # train the stacking model 
    stacking_regressor.fit(X_train, y_train)
    
    # make predictions on test set 
    y_pred = stacking_regressor.predict(X_test_scaled)
    
    # calculate metrics
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    
    print(f"Results for {dim}:")
    print(f"   R² Score: {r2:.4f}")
    print(f"   MAE: {mae:.4f}")
    
    # compare with individual models
    rf_individual = RandomForestRegressor(n_estimators=100, max_depth=4, min_samples_leaf=2, random_state=42)
    pls_individual = PLSRegression(n_components=2)
    
    rf_individual.fit(X_train, y_train)
    pls_individual.fit(X_train, y_train)
    
    y_pred_rf = rf_individual.predict(X_test_scaled)
    y_pred_pls = pls_individual.predict(X_test_scaled)
    if len(y_pred_pls.shape) > 1:
        y_pred_pls = y_pred_pls.flatten()
    
    r2_rf = r2_score(y_test, y_pred_rf)
    r2_pls = r2_score(y_test, y_pred_pls)
    mae_rf = mean_absolute_error(y_test, y_pred_rf)
    mae_pls = mean_absolute_error(y_test, y_pred_pls)
    
    print(f"   RF R^2: {r2_rf:.4f}")
    print(f"   PLS R^2: {r2_pls:.4f}")
    print(f"   Stacking R^2 improvement: {r2 - max(r2_rf, r2_pls):.4f}")
    print(f"   RF MAE: {mae_rf:.4f}")
    print(f"   PLS MAE: {mae_pls:.4f}")
    print(f"   Stacking MAE improvement: {min(mae_rf, mae_pls) - mae:.4f}")
    
    # store results
    results_in_sample[dim] = {
        'states': states,
        'y_true': y_test,
        'y_pred': y_pred,
        'r2': r2,
        'mae': mae}
    
    # store the model
    stacking_models_in_sample[dim] = {
        'model': stacking_regressor,
        'feature_cols': features}


Stacking model for income
Results for income:
   R² Score: 0.8858
   MAE: 3.1034
   RF R^2: 0.8735
   PLS R^2: 0.6585
   Stacking R^2 improvement: 0.0123
   RF MAE: 3.5436
   PLS MAE: 6.2970
   Stacking MAE improvement: 0.4403

Stacking model for health
Results for health:
   R² Score: 0.7761
   MAE: 4.8073
   RF R^2: 0.8422
   PLS R^2: 0.7118
   Stacking R^2 improvement: -0.0662
   RF MAE: 4.0613
   PLS MAE: 5.3771
   Stacking MAE improvement: -0.7461

Stacking model for education
Results for education:
   R² Score: 0.7532
   MAE: 1.9939
   RF R^2: 0.8451
   PLS R^2: 0.5678
   Stacking R^2 improvement: -0.0919
   RF MAE: 1.4137
   PLS MAE: 2.7926
   Stacking MAE improvement: -0.5803

Stacking model for social_security
Results for social_security:
   R² Score: 0.8791
   MAE: 3.7397
   RF R^2: 0.8933
   PLS R^2: 0.7198
   Stacking R^2 improvement: -0.0141
   RF MAE: 3.7416
   PLS MAE: 5.9417
   Stacking MAE improvement: 0.0019

Stacking model for housing
Results for housing:
   R² Scor

In [10]:
# function to save metrics, predictions, plots, and LASSO selected features
def save_rf_results(results_dict, models_dict, folder_name, selected_features_dict=None):
    # create output folder if it doesn't exist
    os.makedirs(folder_name, exist_ok=True)

    # 1. save predictions
    # create a DataFrame with true and predicted values for each dimension
    preds = {'state': next(iter(results_dict.values()))['states']}  # Extract state list from one example
    for dim, res in results_dict.items():
        preds[f'{dim}_actual'] = res['y_true']
        preds[f'{dim}_predicted'] = res['y_pred']

    pd.DataFrame(preds).to_csv(f"{folder_name}/results.csv", index=False)

    # 2. save metrics and LASSO selected features
    rows = []
    for dim, res in results_dict.items():
        features_used = models_dict[dim]['feature_cols']

        # prepare row data
        row_data = {
            'dimension': dim,
            'r2': round(res['r2'], 4),
            'mae': round(res['mae'], 4),
            'n_selected_features': len(features_used),
            'selected_features': ', '.join(features_used)}
        rows.append(row_data)

    pd.DataFrame(rows).to_csv(f"{folder_name}/metrics.csv", index=False)
  
    # 5. save plots of true vs predicted values for each dimension
    for dim, res in results_dict.items():
        y_true = res['y_true']
        y_pred = res['y_pred']

        plt.figure(figsize=(12, 10))
        plt.scatter(
            y_true, y_pred,
            color='royalblue',
            edgecolor='black',
            s=250,
            alpha=0.9)

        # diagonal line for perfect predictions
        min_val = min(min(y_true), min(y_pred))
        max_val = max(max(y_true), max(y_pred))
        plt.plot([min_val, max_val], [min_val, max_val],
                 'r--', linewidth=3, label='Perfect prediction')

        plt.xlabel("CONEVAL's statistics", fontsize=30, fontweight='bold')
        plt.ylabel("Predicted Values", fontsize=30, fontweight='bold')
        
        # title with R² value
        title_text = f"{dim.replace('_', ' ').title()}\nR² = {res['r2']:.3f}"
        
        plt.title(title_text, fontsize=35, fontweight='bold')

        plt.xticks(fontsize=20, fontweight='bold')
        plt.yticks(fontsize=20, fontweight='bold')
        plt.legend(fontsize=25)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()

        plt.savefig(f"{folder_name}/{dim}_plot.png", dpi=300, bbox_inches='tight')
        plt.close()

save_rf_results(results_in_sample, stacking_models_in_sample, folder_name='in_sample_val',
                selected_features_dict=selected_features_in_sample)  

## Out-of-Sample Validation

To assess the model’s **true generalization capacity**, we adopt an **out-of-sample validation strategy**, where training is performed exclusively on 2020 data and predictions are evaluated on 2022.

This approach simulates a realistic scenario in which we aim to **nowcast multidimensional poverty** using recent data, but the official CONEVAL statistics for the current year are not yet available. In such cases, we must rely on available statistics about previous years to train the model and then apply it to newer, unlabeled observations.

Beyond its practical relevance, this setup also provides a meaningful **robustness check**: can a model trained on one year generalize to a different temporal context, especially when social and economic conditions have changed?

This question is particularly important in our setting, as the two years under consideration — 2020 and 2022 — are structurally different. **The year 2020 was dominated by the COVID-19 pandemic**, with widespread disruptions to health systems, labor markets, and mobility. In contrast, **2022 reflects a post-pandemic recovery phase**, where most economic and social indicators began to stabilize. 

By testing our models across such heterogeneous contexts, we can assess whether they are merely fitting year-specific noise, or whether they are capturing **deeper, structural patterns of poverty** that persist despite short-term shocks. In this sense, out-of-sample validation serves not only as a forecast test, but also as a diagnostic tool for evaluating the temporal robustness of the modeling approach.

In [11]:
# SELECT FEATURES WITH LASSO USING ONLY 2020 DATA

# copy the 2020 dataset for scaling
df_full_2020 = data_2020.copy()

# standardize features using only 2020 data
scaler_global_2020 = StandardScaler()
X_full_scaled_2020 = scaler_global_2020.fit_transform(df_full_2020[all_features])

# create a new DataFrame with scaled features (only 2020 data)
df_scaled_2020 = pd.DataFrame(X_full_scaled_2020, columns=all_features, index=df_full_2020.index)
# add the target columns to the scaled DataFrame
for dim, target_col in POVERTY_DIMENSIONS.items():
    df_scaled_2020[target_col] = df_full_2020[target_col]

print(f"Scaled 2020 dataset shape: {df_scaled_2020.shape}")

# feature selection using LASSO on 2020 data only
selected_features_out_sample = {}

for dim, target in POVERTY_DIMENSIONS.items():
    print(f"{dim}")
    
    # use scaled 2020 data only
    X_scaled = df_scaled_2020[all_features].values
    y = df_scaled_2020[target].values
    
    # apply LASSO 
    lasso = Lasso(
        random_state=42,
        alpha=1,  
        max_iter=10000
    ).fit(X_scaled, y)
    
    # select features
    selected_mask = lasso.coef_ != 0
    selected_vars = np.array(all_features)[selected_mask]
    selected_features_out_sample[dim] = selected_vars.tolist()
    
    print(f"Selected {len(selected_vars)} / {len(all_features)} features")

Scaled 2020 dataset shape: (32, 45)
income
Selected 11 / 39 features
health
Selected 10 / 39 features
education
Selected 8 / 39 features
social_security
Selected 14 / 39 features
housing
Selected 17 / 39 features
food
Selected 7 / 39 features


In [12]:
# build the stacking model for out-of-sample validation 
results_out_sample = {}
stacking_models_out_sample = {}

for dim, features in selected_features_out_sample.items():
    
    print(f"\nStacking model for {dim}")
    
    # prepare training data 
    X_train = df_scaled_2020[features].values  
    y_train = df_scaled_2020[POVERTY_DIMENSIONS[dim]].values
    
    # prepare test data 
    X_test_original = df_test[features].values
    y_test = df_test[POVERTY_DIMENSIONS[dim]].values
    states = df_test['state'].values
    
    # scale test data using the 2020-only scaler
    X_test_all_scaled = scaler_global_2020.transform(df_test[all_features])
    
    # select only the features needed for this dimension - according to LASSO selection
    feature_indices = [all_features.index(f) for f in features]
    X_test_scaled = X_test_all_scaled[:, feature_indices]
    
    # define base learners
    base_learners = [
        ('rf', RandomForestRegressor(n_estimators=100, max_depth=4, min_samples_leaf=2, random_state=42)),
        ('pls', PLSRegression(n_components=2))]
    
    # define meta-model
    meta_model = LinearRegression()
    
    # create StackingRegressor 
    stacking_regressor = StackingRegressor(
        estimators=base_learners,
        final_estimator=meta_model,
        cv=5,  
        n_jobs=-1)
    
    # train the stacking model on 2020 data only
    stacking_regressor.fit(X_train, y_train)
    
    # make predictions on 2022 test set
    y_pred = stacking_regressor.predict(X_test_scaled)
    
    # calculate metrics
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    
    print(f"Results for {dim}:")
    print(f"   R² Score: {r2:.4f}")
    print(f"   MAE: {mae:.4f}")
    
    # compare with individual models 
    rf_individual = RandomForestRegressor(n_estimators=100, max_depth=4, min_samples_leaf=2, random_state=42)
    pls_individual = PLSRegression(n_components=2)
    
    rf_individual.fit(X_train, y_train)
    pls_individual.fit(X_train, y_train)
    
    # predict on 2022 test set
    y_pred_rf = rf_individual.predict(X_test_scaled)
    y_pred_pls = pls_individual.predict(X_test_scaled)
    if len(y_pred_pls.shape) > 1:
        y_pred_pls = y_pred_pls.flatten()
    
    r2_rf = r2_score(y_test, y_pred_rf)
    r2_pls = r2_score(y_test, y_pred_pls)
    mae_rf = mean_absolute_error(y_test, y_pred_rf)
    mae_pls = mean_absolute_error(y_test, y_pred_pls)
    
    print(f"   RF R^2: {r2_rf:.4f}")
    print(f"   PLS R^2: {r2_pls:.4f}")
    print(f"   Stacking R^2 improvement: {r2 - max(r2_rf, r2_pls):.4f}")
    print(f"   RF MAE: {mae_rf:.4f}")
    print(f"   PLS MAE: {mae_pls:.4f}") 
    print(f"   Stacking MAE improvement: {min(mae_rf, mae_pls) - mae:.4f}")
    
    # store results
    results_out_sample[dim] = {
        'states': states,
        'y_true': y_test,
        'y_pred': y_pred,
        'r2': r2,
        'mae': mae}
    
    # store model
    stacking_models_out_sample[dim] = {
        'model': stacking_regressor,
        'feature_cols': features}

save_rf_results(results_out_sample, stacking_models_out_sample, folder_name='out_sample_val',
                selected_features_dict=selected_features_out_sample)


Stacking model for income
Results for income:
   R² Score: 0.1905
   MAE: 9.4415
   RF R^2: 0.1278
   PLS R^2: -0.2666
   Stacking R^2 improvement: 0.0627
   RF MAE: 10.1872
   PLS MAE: 12.3105
   Stacking MAE improvement: 0.7456

Stacking model for health
Results for health:
   R² Score: -0.5593
   MAE: 11.4766
   RF R^2: -0.6139
   PLS R^2: -0.3913
   Stacking R^2 improvement: -0.1680
   RF MAE: 11.9381
   PLS MAE: 10.7933
   Stacking MAE improvement: -0.6833

Stacking model for education
Results for education:
   R² Score: 0.3689
   MAE: 3.2688
   RF R^2: 0.2882
   PLS R^2: 0.2226
   Stacking R^2 improvement: 0.0807
   RF MAE: 3.4427
   PLS MAE: 3.5592
   Stacking MAE improvement: 0.1738

Stacking model for social_security
Results for social_security:
   R² Score: 0.3371
   MAE: 8.9727
   RF R^2: 0.2670
   PLS R^2: 0.2785
   Stacking R^2 improvement: 0.0586
   RF MAE: 9.9654
   PLS MAE: 9.5260
   Stacking MAE improvement: 0.5533

Stacking model for housing
Results for housing:
   R

# Construction of the *Social Cohesion* Index (2022) using PCA

Since we do not have a direct target variable to measure social cohesion, we apply an **unsupervised method** based on **Principal Component Analysis (PCA)** to extract a latent index that summarizes the information from a set of proxy variables.

## Data Selection and Preprocessing

We select all variables in the dataset whose names contain the word `"cohesion"`. These are the components we previously constructed from different textual data sources (YouTube, Telegram, News, Google Trends) to capture aspects of social cohesion. Each of these variables reflects a proxy derived from textual analysis, so these are not raw inputs but already-processed indicators explicitly built to inform the social cohesion dimension. 

To ensure comparability across variables and to satisfy PCA assumptions, we standardize the selected features using `StandardScaler`, which centers them around zero and rescales them to unit variance.

## PCA: Extracting the Latent Dimension

We perform a PCA and retain only the **first principal component (PC1)**. This component captures the direction of maximum variance in the standardized feature space. Since all selected features aim to reflect some aspect of social cohesion, PC1 can be interpreted as a **latent index** summarizing the shared signal across them.

We chose this approach because:

- By reducing dimensionality, we capture the dominant underlying pattern in a single score.
- Since we don't have a target for this dimension, our possibilities were limited but we still believe this approach allows to define weights in a robust and non-arbitrary way
- Reflects the empirical correlation structure of the data, which is appropriate in the absence of a predefined ground truth.

## Handle the Sign

One limitation of PCA is that the sign of the components is not uniquely identified — multiplying all loadings and scores by -1 yields an equivalent solution.

To ensure consistency in interpretation (i.e., **higher scores mean worse social cohesion**), we compute the **sum of the loadings** for the first component:
- If the sum is **negative**, it means that high raw scores are associated with **better** social cohesion. In this case, we **invert the sign** of the PCA scores so that higher values reflect **higher deprivation**.
- If the sum is **positive**, we keep the scores as they are.

This correction guarantees that a value of 100 consistently means "lowest social cohesion" — in line with the interpretation of other poverty indicators.

## Normalization to a 0–100 Scale

After sign correction, the scores are normalized to a 0–100 scale using `MinMaxScaler`. This is aligned with the conventions adopted by CONEVAL and our other dimensional estimates, which express deprivation as a **percentage of the population** affected.

The resulting index can thus be interpreted as a relative measure of social cohesion deprivation, comparable across Mexican states.

## Observed Anomalies: 5 States with Extreme Values

In the final normalized scores, we observe the presence of unrealistic values:
- One state receives a score of exactly **0**.
- Four states have scores **above 80**, with one state reaching exactly **100**.

While these results are not computationally incorrect, they may appear questionable — especially given that we do not observe such extreme values in any of the other dimensions. However, several technical factors can explain this behavior:

- **Outliers in the original features**: some states may exhibit near-zero values across multiple cohesion-related components, particularly in cases where textual data coverage was sparse or unbalanced. This can drive their PCA score toward the lower bound of the distribution.

- **High-leverage observations**: PCA is inherently sensitive to atypical combinations of feature values. A single state with an unusual profile — even if not extreme in any single component — can strongly influence the orientation of the principal component and receive a disproportionately high or low score.

- **Scaling effects**: the use of `MinMaxScaler` maps the minimum and maximum PCA scores to 0 and 100 by design. As a result, there will always be at least one state assigned **0** and one assigned **100**, regardless of how realistic those values are in substantive terms.

Ultimately, these extreme scores should be interpreted as **relative positions**: they indicate how a state ranks within the empirical distribution of social cohesion *as captured by the PCA*, rather than representing absolute levels of deprivation. A score of 100 does not imply that 100% of the population experiences cohesion poverty — it simply reflects that the state ranks at the very bottom in relative terms.

To avoid producing exact 0 and 100 values, one option would have been to restrict the output range during normalization, for example:

```python
scaler_pct = MinMaxScaler(feature_range=(5, 80))
````

However, we opted not to implement this adjustment, for two main reasons:

- The choice of an alternative range would have been arbitrary and thus methodologically debatable;
- Compressing the score range would have reduced the spread of the values, limiting the model's ability to differentiate between intermediate cases.

We therefore retained the full [0, 100] scale, acknowledging that the extremes reflect model mechanics as much as underlying variance in the data.

In [13]:
# select social cohesion features
social_features = [col for col in data_2022.columns if 'cohesion' in col.lower()]
X_social_2022 = data_2022[social_features].dropna()

# standardize the features
scaler = StandardScaler()
X_scaled_2022 = scaler.fit_transform(X_social_2022)

# PCA to extract the first component
pca = PCA(n_components=1)
social_cohesion_score_2022 = pca.fit_transform(X_scaled_2022)

# get and print loadings 
loadings = dict(zip(social_features, pca.components_[0]))
print("PCA loadings (PC1):")
for feature, weight in loadings.items():
    print(f"{feature}: {weight:.3f}")

# invert the sign of the PCA scores if necessary
# (this is to ensure that a higher score indicates worse cohesion)
# (if the sum of loadings is negative, invert the sign to have: 100 = worst cohesion)
if np.sum(pca.components_[0]) < 0:
    print(" inverting sign of PCA scores to have: 100 = worse cohesion")
    social_cohesion_score_2022 = -social_cohesion_score_2022

# normalize the scores to a 0-100 scale
scaler_pct = MinMaxScaler(feature_range=(0, 100))
social_cohesion_normalized = scaler_pct.fit_transform(social_cohesion_score_2022)

# final df 
cohesion_df = data_2022.loc[X_social_2022.index, ['state', 'year']].copy()
cohesion_df['social_cohesion_score'] = social_cohesion_normalized

# save 
os.makedirs("PCA", exist_ok=True)
cohesion_df.to_csv("PCA/score.csv", index=False)

cohesion_df.head(10)

PCA loadings (PC1):
cohesion_gt: -0.382
social_cohesion_avg_sentiment: -0.373
social_cohesion_pct_yt: 0.619
social_cohesion_pct_tg: 0.576


,state,year,social_cohesion_score
0,Aguascalientes,2022,34.618119
1,Baja California,2022,88.612437
2,Baja California Sur,2022,43.232871
3,Campeche,2022,16.569366
4,Coahuila,2022,41.210944
5,Colima,2022,31.376835
6,Chiapas,2022,92.302337
7,Chihuahua,2022,24.930064
8,Ciudad de México,2022,24.701380
9,Durango,2022,36.179750
